### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

In [2]:
from pymystem3 import Mystem

In [3]:
from collections import Counter

In [4]:
df = pd.read_csv('/datasets/data.csv')

In [5]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [6]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [8]:
df['children'] = df['children'].apply(abs)

In [9]:
df['days_employed'] = df['days_employed'].apply(abs)

In [10]:
print(df['children'].value_counts()) 

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


In [11]:
def conversion_children_count(df):
    if df['children'] == 20:
        return 2
    else:
        return df['children']
 
 
df['children'] = df.apply(conversion_children_count, axis=1)

### Вывод

1. После применения метода df.describe() видно, что в данных есть аномальные / "необычные" значения: 

1) 20 детей у нескольких заемщиков 
2) минимальное значение в 'children' -1 ребенок 
3) Отрицательные значения в столбце days_employed, где указано количество дней стажа и среднее значение 63046 дня,  то есть 172,7 лет стажа.

2. После применения метода df.info() видно, что есть пропущенные значения в столбцах days_employed (количество дней стажа) и total_income (доход). Возможно, это техническая ошибка или клиент по каким-то причинам решил скрыть эти данные. Мы не можем оставить эти значения не заполненными, так как нам нужно создать модель кредитного скоринга, а пропуски могут исказить окончательное решение - одобрить кредит или нет.

3.Для удобства дальнейшей работы сразу заменила отрицательные значения на значения по модулю.

4. Скорее всего, 20 детей у нескольких заемщиков (76 человек) это техническая ошибка при вводе, к тому же нет заемщиков с 5+ количеством детей, написала функцию, которая заменяет "20" на "2".

5. В оригинальном датафрейме значения в столбцах 'days_employed' и 'total_income' дробные, это вещественный тип данных float64 (видно из метода info(), что неудобно для восприятия и дальнейшей работы - заменим на целые числа.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [12]:
print(df[df['days_employed'].isnull()].count()) 

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


В 2 174 строках нет значений в столбцах 'days_employed' и 'total_income'.

In [13]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median')) 

In [14]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Так как метод describe() показал довольно большой разброс между минимальным и максимальным значениями в столбцах
'days_employed' и 'total_income', то мы не можем заменить пропуски средним значением по столбцу, это будет слишком
грубая оценка, нужно заменить пропуски на медиану. Запускаем метод info(), все ячейки теперь заполнены.

### Замена типа данных

In [16]:
df['days_employed'] = df['days_employed'].astype(int)

In [17]:
df['total_income'] = df['total_income'].astype(int)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Так как нам нужно перевести вещественные числа в целые, применяем метод astype() и параметр int - integer(целое число), метод to_numeric() нам не подойдет, потому что после его применения числа получают тип данных float. После применения метода info() видим, что в нужных столбцах тип данных поменялся на int64.

### Обработка дубликатов

In [19]:
df.duplicated().sum()

54

Применила стандартный метод duplicated для поиска дубликатов, нашла их общее количество. Ниже вывела их на экран, выглядят действительно как дубли данных. Возможные причины появления дубликатов: техническая ошибка/ задвоение данных или заемщик подавал заявку на кредит несколько раз, указывал разные цели. Дубликаты удаляют потому, что они могут исказить финальный итог нашего исследования.

In [20]:
df[df.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1005,0,365213,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,ремонт жилью
1191,0,365213,61,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции с недвижимостью
1511,0,365213,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
1681,0,365213,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
2052,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
10697,0,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,сыграть свадьбу
13878,1,1547,31,среднее,1,женат / замужем,0,F,компаньон,0,172357,покупка жилья
17379,0,1547,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью
17774,1,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости


In [21]:
df = df.drop_duplicates()

In [22]:
df.duplicated().sum()

0

### Вывод

Удалила дубликаты с помощью стандартного метода drop_duplicates() и проверила, что они были удалены из датафрейма, вызвав метод df.duplicated().sum().

### Лемматизация

In [23]:
m = Mystem() 

df['lemmas'] = df['purpose'].apply(m.lemmatize)
#print(df['lemmas'])
print(Counter(m.lemmatize(' '.join(df['purpose']))))






/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Counter({' ': 55066, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'подержать': 853, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


### Вывод

### Категоризация данных

Категоризация по столбцу 'children'

In [24]:
print(df['children'].value_counts())

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


In [25]:
def amount_children_group(children):
        
        if children < 1:
                return 'нет детей'
        if children > 2:
                return 'многодетные'
        return '1-2 ребенка' 

In [26]:
print(amount_children_group(4)) 

многодетные


In [27]:
df['amount_children_group'] = df['children'].apply(amount_children_group) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Категоризация по доходам

Исходя из того, что мы знаем квантили распределения по доходу (из метода describe()), разобьем значения в столбце 'total_income' на 4 равные группы с помощью метода qcut.  Этот метод делит совокупность на равные группы, в отличие от метода cut

In [28]:
#df['income_amount_group'] = pd.qcut(df['total_income'], 4)

In [29]:
#df.groupby('income_amount_group')['debt'].agg(['count', 'mean'])

**После применения метода groupby видим, что у нас есть 4 примерно одинаковых группы по величине дохода, каждая по 5200-5400 человек.
Напишем функцию, в которую передадим значения этих 4 групп по уровню дохода**.

In [30]:
def total_income_group(total_income):
        
        if 20000 <= total_income < 110000:
                return 'доход 20 000-110 000'
        if 110000 <= total_income < 150000:
                return 'доход 110 000-150 000'
        if 150000 <= total_income <= 200000:
                return 'доход 150 000-200 000'
        if total_income > 200000:    
                return 'доход выше 200 000' 

In [31]:
print(total_income_group(250000)) 

доход выше 200 000


In [32]:
df['total_income_group'] = df['total_income'].apply(total_income_group) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий ревьюера - 2</h1>
Хорошо
</div>

In [33]:
print(df.head(10))

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   Среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   СРЕДНЕЕ             1   
8         2           6929         35    ВЫСШЕЕ             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

Категоризация по цели кредита

При помощи Counter мы получили следующий список лемм:

Counter({' ': 55066, 
'недвижимость': 6353,
'покупка': 5900, 
'жилье': 4461, 
'автомобиль': 4308,
'образование': 4014, 
'с': 2918,
'операция': 2604, 
'свадьба': 2335,
'свой': 2231, 
'на': 2228,
'строительство': 1879,
'высокий': 1374, 
'получение': 1315,
'коммерческий': 1312,
'для': 1290,
'жилой': 1231, 
'сделка': 941,
'дополнительный': 907,
'заниматься': 904,
'подержать': 853, 
'проведение': 773,
'сыграть': 769, 
'сдача': 652, 
'семья': 638,
'собственный': 635,
'со': 627,
'ремонт': 607,
'приобретение': 461,
'профильный': 436,
'подержанный': 111,
'\n': 1})

Выделим наиболее часто встречаемые леммы 'недвижимость' (6353), 'жилье' (4461), 'автомобиль' (4308),'образование' (4014), 'свадьба' (2335) и создадим функцию для классификации целей кредита. С появлением столбца 'lemmas' можем быстрее найти цель кредита по ключевым словам.

In [34]:
def purpose_simplification(text):
    if 'автомобиль' in text:
        return 'авто'
    if 'свадьба' in text:
        return 'свадьба'
    if 'образование' in text:
        return 'образование'
    if 'жилье' or 'недвижимость' in text:
        return 'ком. недвижимость'
df['lemmas'] = df['lemmas'].apply(purpose_simplification)
print(df.head(10))

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   Среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   СРЕДНЕЕ             1   
8         2           6929         35    ВЫСШЕЕ             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


### Вывод

Я разбила на категории данные в столбцах 'children' и 'total_income'.

1.Из датафрейма видно, что у заемщиков может быть от 0 до 5 детей, поэтому для удобства работы разобьем заемщиков на группы, в зависимости от количества детей. После создания и применения функции def amount_children_group(children) получила следующие группы: "нет детей", "1-2 ребенка", "многодетные". Проверила, что функция работает при помощи вывода print(amount_children_group(количество детей)) и добавила столбец 'amount_children_group' к датафрейму.

2. Значения в столбце 'total_income' с помощью функции total_income_groupразбила на 4 категории дохода - 1) от 20 000 до 110 000, 2) от 110 000 до 150 000 3) от 150 000 до 200 000 4) больше 200 000.  Проверила, что функция работает при помощи вывода print(total_income_group(сумма дохода)) и добавила столбец 'amount_income_group' к датафрейму.

3. Данные разбиты на категории, исходя из общепринятых представлений о многодетности и уровню дохода в крупных городах.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
children_debt_pivot = df.pivot_table(index = ['amount_children_group'], values = 'debt')

In [36]:
print(children_debt_pivot)

                           debt
amount_children_group          
1-2 ребенка            0.092640
многодетные            0.081579
нет детей              0.075353


### Вывод

Построим сводную таблицу для ответа на этот вопрос. По соотношению между категориями заемщиков по количеству детей видно, что заемщики, не имеющие детей, менее склонны к просрочке платежа - вероятность 7,5%. А более склонны к просрочке платежа те, у кого есть 1-2 ребенка - 9,2%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
family_status_debt_pivot = df.pivot_table(index = ['family_status'], values = 'debt')

In [38]:
print(family_status_debt_pivot)

                           debt
family_status                  
Не женат / не замужем  0.097509
в разводе              0.071130
вдовец / вдова         0.065693
гражданский брак       0.093202
женат / замужем        0.075421


### Вывод

Из сводной таблицы видим соотношение, что менее всего склонны к просрочкам платежей вдовцы и вдовы - 6,6%, разведенные - 7,1%, и состоящие в официальном браке - 7,5%; более всего - не находящиеся в браке - 9,8% и живущие в гражданском браке - 9,3%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
total_income_group_debt_pivot = df.pivot_table(index = ['total_income_group'], values = 'debt')

In [40]:
print(total_income_group_debt_pivot)

                           debt
total_income_group             
доход 110 000-150 000  0.087225
доход 150 000-200 000  0.084977
доход 20 000-110 000   0.080645
доход выше 200 000     0.070653


### Вывод

Исходя из результатов сводной таблицы видно, что наименее склонны к просрочкам по кредиту заемщики с доходом выше 200 000 - 7% и заемщики с доходом 20 000 - 110 000. А вероятность просрочки среди клиентов с доходами в интервале 110 000 - 200 000 примерно одинакова - 8,4% и 8,7%.

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
lemmas_debt_pivot = df.pivot_table(index = ['lemmas'], values = 'debt')

In [42]:
print(lemmas_debt_pivot)

                       debt
lemmas                     
авто               0.093547
ком. недвижимость  0.072314
образование        0.092177
свадьба            0.079657


### Вывод

Самая низкая вероятность просрочки у тех, кто брал кредит на недвижимость - 7,2%, это можно объяснить тем, что суммы по таким кредитам большие и требуют наличия высокого дохода/ уверенности в доходе на многие годы. Самая большая вероятность просрочки у тех, кто брал кредит на автомобиль - 9,3%, затем - у тех, кто взял кредит на образование - 9,2%. Интересно, что те, кто брал кредит на свадьбу, возвращает его с большой вероятностью.

### Шаг 4. Общий вывод

Так как в начале нашей работы говорилось, что результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку, опишем характеристики "идеального" клиента:
1. У него нет детей.
2. Он состоит в официальном браке сейчас или когда-то был (вдовцы и разведенные).
3. Его доход составляет или более 200 000 или сумму в интервале 20 000-110 000.
4. Скорее всего, целью кредита будет приобретение недвижимости.